From our nlp course project
https://github.com/Aaalan-Zhang/nlp-from-scratch-assignment-fall2024/blob/main/crawler/tests/0_webpage_parsing.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.listdir("./drive/MyDrive/Proj/News_fetcher")

os.chdir("./drive/MyDrive/Proj/News_fetcher")

Mounted at /content/drive


# Crawler

In [ ]:
# # === Install Chrome and ChromeDriver v137 ===
# !apt-get remove -y chromium-browser google-chrome-stable || true
# !rm -rf /usr/bin/google-chrome /usr/bin/chromedriver137

# !wget -q https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.119/linux64/chrome-linux64.zip
# !unzip -q chrome-linux64.zip
# !mv chrome-linux64/chrome /usr/bin/google-chrome

# !wget -q https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.119/linux64/chromedriver-linux64.zip
# !unzip -q chromedriver-linux64.zip
# !mv chromedriver-linux64/chromedriver /usr/bin/chromedriver137
# !chmod +x /usr/bin/google-chrome /usr/bin/chromedriver137

In [ ]:
%pip install -q google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
typeguard 4.4.3 requires typing_extensions>=4.14.0, but you have typing-extensions 4.13.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_mach

In [ ]:
!pip install tqdm

## Original Crawler from 711

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import time
# import random
# import warnings
# from urllib.parse import urljoin
# from urllib3.exceptions import InsecureRequestWarning

# user_agents = [
#     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
#     'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
#     'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
# ]
# warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# # Define headers for requests
# headers = {'User-Agent': random.choice(user_agents)}
# session = requests.Session()

# # Fetch the page text with retry mechanism
# def fetch_page_text(url, retries=3, timeout=5):
#     attempt = 0
#     while attempt < retries:
#         try:
#             # Send a GET request to the URL with a timeout
#             response = session.get(url, timeout=timeout, headers=headers, verify=False)
#             response.raise_for_status()  # Raise an error for bad responses

#             # Parse the page content
#             soup = BeautifulSoup(response.content, 'lxml')

#             # Extract and return the text from the page and the soup object
#             return soup.get_text(separator='\n', strip=True), soup

#         except requests.exceptions.RequestException as e:
#             attempt += 1
#             time.sleep(1)  # Optional: wait before retrying

#         if attempt == retries:
#             return None, None

# # Extract all sublinks (anchor tags) and handle relative URLs
# def extract_sublinks(soup, base_url):
#     sublinks = []
#     for link in soup.find_all('a', href=True):
#         sublink = link['href']
#         # Convert relative URLs to absolute URLs using urljoin
#         full_url = urljoin(base_url, sublink)
#         sublinks.append(full_url)
#     return sublinks

# # Crawl through the URLs and fetch the data with retry
# def crawl_urls(url_list):
#     results = {}
#     sublinks_data = []  # List to store (parent_url, sublink) pairs
#     for index, url in enumerate(url_list):
#         print(f"Fetching: {url}")
#         text, soup = fetch_page_text(url)
#         if text and soup:
#             # Store the crawled text
#             results[url] = text
#             # Extract sublinks
#             sublinks = extract_sublinks(soup, url)
#             for sublink in sublinks:
#                 sublinks_data.append((url, sublink))
#         else:
#             print(f"Failed to parse URL at index: {index}, URL: {url}")
#     return results, sublinks_data

# # Save the crawled text data to .txt files
# def save_crawled_data(crawled_data, urls):
#     for index, url in enumerate(urls):
#         # Fetch the text corresponding to the URL from the crawled_data dictionary
#         text = crawled_data.get(url, "")

#         # Remove all newline characters from the text
#         cleaned_text = text.replace('\n', ' ')

#         # Define the file name with the index from the URLs list
#         output_file = f"/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_text_data/{index}.txt"

#         # Save the cleaned text to the file
#         with open(output_file, 'w', encoding='utf-8') as file:
#             file.write(cleaned_text)

# # Save sublinks to a CSV file
# def save_sublinks_to_csv(sublinks_data, output_csv):
#     df = pd.DataFrame(sublinks_data, columns=['Parent URL', 'Sublink'])
#     df.to_csv(output_csv, index=False)

# if __name__ == "__main__":
#     file_path = '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/raw/raw_csv_data/data_source.csv'
#     data = pd.read_csv(file_path)

#     # Extract non-empty URLs from the 'Source URL' column
#     urls = data[data['Select'] == 'Webpage']['Source URL'].dropna().unique()

#     # Start crawling the URLs
#     crawled_data, sublinks_data = crawl_urls(urls)

#     # Save the crawled text data
#     save_crawled_data(crawled_data, urls)

#     # Save sublinks to a CSV file
#     sublinks_output_file = "/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_sublinks.csv"
#     save_sublinks_to_csv(sublinks_data, sublinks_output_file)

#     print("Crawling complete!")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/raw/raw_csv_data/data_source.csv'

## 参考消息

In [ ]:
from datetime import datetime
url = "https://www.cankaoxiaoxi.com/#/detailsPage/tt/329d19f3dfae4b50b524f428cd62637c/1/2025-06-18%2009:46?childrenAlias=undefined"
prefix = "/content/drive/MyDrive/Proj/News_fetcher/"


# Get today's date
suffix = datetime.today().strftime("%Y-%m-%d")
suffix

'2025-06-19'

### Main

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import time
# import random
# import warnings
# from urllib.parse import urljoin
# from urllib3.exceptions import InsecureRequestWarning
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup
# import time

# user_agents = [
#     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
#     'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
#     'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
# ]
# warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# # Define headers for requests
# headers = {'User-Agent': random.choice(user_agents)}
# session = requests.Session()

# # Fetch the page text with retry mechanism
# def fetch_page_text(url, retries=3, timeout=5):
#     attempt = 0
#     while attempt < retries:
#         try:
#             # Send a GET request to the URL with a timeout
#             response = session.get(url, timeout=timeout, headers=headers, verify=False)
#             response.raise_for_status()  # Raise an error for bad responses

#             # Parse the page content
#             soup = BeautifulSoup(response.content, 'lxml')
#             print("response is :")
#             print(response.content)
#             # Extract and return the text from the page and the soup object
#             return soup.get_text(separator='\n', strip=True), soup

#         except requests.exceptions.RequestException as e:
#             attempt += 1
#             time.sleep(1)  # Optional: wait before retrying

#         if attempt == retries:
#             return None, None

# # def fetch_page_text_selenium(url, retries=3, timeout=5):
# #     # Setup headless Chrome
# #     options = Options()
# #     options.add_argument('--headless')
# #     options.add_argument('--disable-gpu')

# #     driver = webdriver.Chrome(options=options)

# #     # Open page
# #     driver.get("https://your_url_here.com")
# #     time.sleep(3)  # wait for JS to load; adjust as needed

# #     # Get rendered HTML
# #     html = driver.page_source
# #     soup = BeautifulSoup(html, 'lxml')

# #     # Now you can extract dynamically loaded content
# #     print(soup.prettify())

# #     driver.quit()

# # Extract all sublinks (anchor tags) and handle relative URLs
# def extract_sublinks(soup, base_url):
#     sublinks = []
#     for link in soup.find_all('a', href=True):
#         sublink = link['href']
#         # Convert relative URLs to absolute URLs using urljoin
#         full_url = urljoin(base_url, sublink)
#         sublinks.append(full_url)
#     return sublinks

# # Crawl through the URLs and fetch the data with retry
# def crawl_urls(url_list):
#     results = {}
#     sublinks_data = []  # List to store (parent_url, sublink) pairs
#     for index, url in enumerate(url_list):
#         print(f"Fetching: {url}")
#         text, soup = fetch_page_text(url)
#         if text and soup:
#             # Store the crawled text
#             results[url] = text
#             # Extract sublinks
#             sublinks = extract_sublinks(soup, url)
#             for sublink in sublinks:
#                 sublinks_data.append((url, sublink))
#         else:
#             print(f"Failed to parse URL at index: {index}, URL: {url}")
#     return results, sublinks_data

# # Save the crawled text data to .txt files
# def save_crawled_data(crawled_data, urls, prefix):
#     for index, url in enumerate(urls):
#         # Fetch the text corresponding to the URL from the crawled_data dictionary
#         text = crawled_data.get(url, "")

#         # Remove all newline characters from the text
#         cleaned_text = text.replace('\n', ' ')

#         # Define the file name with the index from the URLs list
#         output_file = f"{prefix}data/crawled_data/crawled_text_data/{index}.txt"
#         output_dir = os.path.dirname(output_file)

#         # Check if directory exists, and create if not
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)

#         # Save the cleaned text to the file
#         with open(output_file, 'w', encoding='utf-8') as file:
#             file.write(cleaned_text)

# # Save sublinks to a CSV file
# def save_sublinks_to_csv(sublinks_data, output_csv):
#     df = pd.DataFrame(sublinks_data, columns=['Parent URL', 'Sublink'])
#     df.to_csv(output_csv, index=False)

# if __name__ == "__main__":
#     # file_path = '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/raw/raw_csv_data/data_source.csv'
#     # data = pd.read_csv(file_path)

#     # # Extract non-empty URLs from the 'Source URL' column
#     # urls = data[data['Select'] == 'Webpage']['Source URL'].dropna().unique()

#     url = "https://www.cankaoxiaoxi.com/#/detailsPage/tt/329d19f3dfae4b50b524f428cd62637c/1/2025-06-18%2009:46?childrenAlias=undefined"
#     prefix = "/content/drive/MyDrive/Proj/News_fetcher/"

#     urls = [url]
#     # Start crawling the URLs
#     crawled_data, sublinks_data = crawl_urls(urls)

#     # Save the crawled text data
#     save_crawled_data(crawled_data, urls, prefix)

#     # Save sublinks to a CSV file
#     sublinks_output_file = prefix + "data/crawled_data/crawled_sublinks.csv"
#     save_sublinks_to_csv(sublinks_data, sublinks_output_file)

#     print("Crawling complete!")

### Workable for 参考消息


In [ ]:
def wait_for_stable_page(driver, timeout=10, interval=1.5):
    """Waits for the whole text content to stop changing."""
    stable_counter = 0
    prev_text = ""
    start_time = time.time()

    while time.time() - start_time < timeout:
        try:
            html = driver.page_source
            curr_text = html.strip()
        except:
            curr_text = ""

        if curr_text == prev_text and len(curr_text) > 0:
            stable_counter += 1
            if stable_counter >= 2:  # Stable twice in a row
                return curr_text
        else:
            stable_counter = 0
        prev_text = curr_text
        time.sleep(interval)
        # print(len(curr_text))
    return
    raise TimeoutException("detailsPage content did not stabilize in time.")
import time
from selenium.common.exceptions import TimeoutException

def wait_for_stable_page(driver, timeout=20, interval=1.5, tolerance=0.05):
    """
    Waits for the whole text content to stop changing significantly.
    If the change between two versions is <1% for 3 times in a row, it's considered stable.
    """
    stable_counter = 0
    prev_text = ""
    start_time = time.time()

    def similarity_ratio(text1, text2):
        if not text1 or not text2:
            return 0.0
        len1, len2 = len(text1), len(text2)
        min_len = min(len1, len2)
        diff = abs(len1 - len2) + sum(c1 != c2 for c1, c2 in zip(text1, text2))
        return 1.0 - diff / max(len1, len2)

    while time.time() - start_time < timeout:
        try:
            html = driver.page_source
            curr_text = html.strip()
        except:
            curr_text = ""

        if prev_text and similarity_ratio(prev_text, curr_text) >= (1 - tolerance):
            stable_counter += 1
            if stable_counter >= 3:
                return curr_text
        else:
            stable_counter = 0

        prev_text = curr_text
        time.sleep(interval)
    return
    raise TimeoutException("Page content did not stabilize in time.")

def safe_get(driver, url, max_retries=5, wait_between=3):
    for attempt in range(max_retries):
        try:
            driver.get(url)
            return  # success
        except TimeoutException:
            print(f"[Attempt {attempt+1}] Timeout loading: {url}")
            time.sleep(wait_between)
        except (WebDriverException, InvalidSessionIdException) as e:
            print(f"[{attempt+1}] WebDriver error: {e}. Restarting driver...")
            try:
                driver.quit()
            except Exception:
                pass
            driver = get_fresh_driver()
            time.sleep(2)
    raise TimeoutException(f"Failed to load {url} after {max_retries} attempts.")

def get_fresh_driver():
    try:
        print("Initializing Chromedriver")
        driver = gs.Chrome()
        time.sleep(2)  # Let browser settle
        return driver
    except Exception as e:
        print("Driver failed to initialize:", e)
        return None

In [ ]:
def restart_driver_and_get(url, retries=3):
    for attempt in range(retries):
        driver = get_fresh_driver()
        if driver is None:
            continue
        try:
            driver.get(url)
            return driver
        except WebDriverException as e:
            print(f"[{attempt+1}] Error loading {url}: {e}")
            try:
                driver.quit()
            except:
                pass
            time.sleep(2)
    print("All attempts failed.")
    return None

In [ ]:
import pandas as pd

# Assume these are your lists
# sublinks = ['https://example.com/page1', 'https://example.com/page2', ...]
# contents = ['Text content of page 1...', 'Text content of page 2...', ...]

# Combine into a DataFrame
def save_to_csv(sublinks, contents, htmls, file_name, tags=None):
    df = pd.DataFrame({
        'ind': range(len(sublinks)),
        'url': sublinks,
        'content': contents,
        'html': htmls,
    })
    if tags:
      df['tag'] = tags
    # Save to CSV
    df.to_csv(f'{prefix}data/crawled_data/{suffix}_{file_name}', index=False, encoding='utf-8-sig')  # Use utf-8-sig for Chinese
def save_lists_to_csv(file_name, **columns):
    """
    Save multiple equal-length lists to a CSV file.

    Parameters:
    - file_name: str, the output CSV file path
    - columns: keyword arguments where key is column name and value is a list of column data
    """
    # Get the length of the first column
    lengths = [len(col) for col in columns.values()]
    if not all(length == lengths[0] for length in lengths):
        raise ValueError("All input lists must have the same length.")

    # Create DataFrame and save
    df = pd.DataFrame(columns)
    df.to_csv(f'{prefix}data/crawled_data/{suffix}_{file_name}', index=True, encoding='utf-8-sig')
    print(f"CSV saved to {prefix}data/crawled_data/{suffix}_{file_name}")
def load_csv(file_name):
    return pd.read_csv(f'{prefix}data/crawled_data/{suffix}_{file_name}', encoding='utf-8-sig')

### Index Page

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    WebDriverException,
    TimeoutException,
    NoSuchElementException,
    InvalidSessionIdException
)
from bs4 import BeautifulSoup
import time
import google_colab_selenium as gs

In [ ]:
index_url = "https://www.cankaoxiaoxi.com/#/index"
sublinks = []
contents = []
htmls = []


# # Parse after content loads
# html = driver.page_source
# soup = BeautifulSoup(html, "lxml")

# # Print readable content
# print("Title:", driver.title)
# print(soup.get_text()[:10000])  # preview 1000 chars of text

cankaoxiaoxi_index_visiting(sublinks, contents, htmls, index_url)
# driver.quit()
# save_to_csv(sublinks, contents, "index_contents.csv")
save_lists_to_csv("index_contents.csv", sublink = sublinks, content = contents, html = htmls)

Initializing Chromedriver


<IPython.core.display.Javascript object>

参考消息
参考消息第一关注中国国际观点锐参考体育健康科技应用文化旅游参考漫谈参考智库军事民族品牌参考人物产经地方译名文娱我的位置：首页>新闻详情梦舟飞船成功进行零高度逃逸试验2025-06-19 14:51:1331331分享到：用微信扫描二维码6月17日，我国在酒泉卫星发射中心成功组织实施梦舟载人飞船零高度逃逸飞行试验，标志着我国载人月球探测工程研制工作取得新的重要突破。（新华社发）参考消息网6月19日报道 据法哈拉斯新闻网6月17日报道，中国正通过太空技术领域的重大进展推进其探月雄心。据中国载人航天工程办公室消息，中国17日成功组织实施梦舟载人飞船零高度逃逸飞行试验，这是其实现航天员登月目标的关键一步。报道称，此次测试涉及飞船的逃逸塔系统，通过点燃固体火箭发动机使船塔组合体升空。最终，梦舟载人飞船安全返回地面，验证了其系统安全性。这是否意味着这一次中国将先于美国国家航空航天局(NASA)登月？报道称，中国梦舟载人飞船零高度逃逸飞行试验的成功，标志着其月球探索进程中的一个关键节点，使其有望与NASA的阿耳忒弥斯登月计划展开竞争。报道称，中国在月球技术领域的进步凸显了太空探索领域的竞争格局，中美两国正力争在未来任务中占据主导地位。另据印度新闻9直播网站6月17日报道，中国成功进行梦舟载人飞船发射台中止试验。这是在酒泉卫星发射中心进行的零高度逃逸飞行试验。报道称，这是继1998年神舟载人飞船零高度逃逸飞行试验之后，时隔27年中国再度组织实施这项试验。神舟飞船用于中国空间站乘组人员轮换。经过重新设计的梦舟载人飞船逃逸发动机正常运行，船塔组合体在固体发动机推进下升空，返回舱与逃逸塔安全分离。在飞行两分钟后，返回舱使用气囊缓冲方式安全着陆于试验落区预定区域。报道指出，这一安全机制对载人航天至关重要，因为一旦飞行过程中发生故障，它可以使乘员迅速远离危险并安全返回地面。报道称，此次试验的成功为中国后续载人月球探测任务奠定了重要的技术基础。中国的载人月面着陆器被命名为“揽月”。由于大气压力的变化，乘组人员的逃逸设备会根据不同的高度进行调整。乘组人员逃逸系统的火箭发动机位于飞船的顶部。梦舟载人飞船系统全面负责逃逸与救生两项任务。报道称，中国一直在测试其地月空间运输体系各个组成部分，并加强了载人登月计划。（编译/邬眉、马丹）更多新闻梦舟飞船成功进行零高度逃逸试验2025-06-19 

<IPython.core.display.Javascript object>

参考消息
参考消息第一关注中国国际观点锐参考体育健康科技应用文化旅游参考漫谈参考智库军事民族品牌参考人物产经地方译名文娱我的位置：首页>中国>新闻详情法媒：中国治理长江水质取得显著成效2025-06-19 17:23:0622624分享到：用微信扫描二维码参考消息网6月19日报道 法国《回声报》网站6月16日刊登记者拉斐尔·巴莱涅里撰写的题为《在中国，治理长江污染是一项艰巨浩大的工程》的报道，编译如下：这是一座依山而建的县城，长江游轮经过这里，在蜿蜒的江水上航行。在这个初夏的美好日子里，巫山码头的游轮载满游客。巫山县这座中国西南部的城镇就坐落在亚洲最长河流长江那壮观的峡谷中。旁边，几艘颜色鲜亮、样式有些特别的蓝色小船也正等待解缆起航。自2003年起，巫山便配备了一支由数十艘清漂作业船组成的船队。它们肩负的使命是清理长江，每天出发“捕捞”污染长江的垃圾，包括塑料、树枝、碎木片……每年5月至10月的汛期是最为棘手的阶段，季风带来的暴雨将周边城市和山上的各类垃圾冲入长江。得益于船上安装的自动过滤系统，该清污工程每年可打捞处理2万余吨垃圾。巫山县为治理其管辖范围内55公里长的长江干流江段，每年投入上百万元人民币的专项资金。在这项浩大工程启动20年后，巫山县开始收获治理成效。根据中国水质标准体系，长江巫山段干流断面水质已连续8年保持Ⅱ类标准、总磷浓度连续11年下降。中国拥有1500多条流域面积超过1000平方公里的河流和七大水系。因此治理水污染问题对中国至关重要。据了解，长江流域拥有中国35%的水资源量。超过4亿中国人生活在长达6300公里的长江沿岸。根据世界自然基金会的数据，长江流域还提供了全国一半以上的淡水鱼和三分之二的水稻产量。过去40年，快速的城镇化进程和狂飙突进的经济发展等都给长江流域的自然生态系统造成了严重影响。然而问题远不止于长江。根据2014年《自然》周刊发表的一篇文章，中国全国范围内的水污染每年导致大量水资源无法利用。在此背景下，中国自新世纪以来不断加大长江治理力度。2002年，中国颁布《中华人民共和国水法》，标志着中国水治理的重要转折。随后在2015年，中国制定阶段性目标：到2020年七大重点流域水质优良(达到或优于Ⅲ类)比例总体达到70%以上，到2030年，全国七大重点流域水质优良比例总体达到75%以上。为此，中国采取了一系列有力措施：关停沿江污染企业、

In [ ]:
# save_to_csv(sublinks, contents, htmls, "index_contents_temp.csv")

### generalColumns


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    WebDriverException,
    TimeoutException,
    NoSuchElementException,
    InvalidSessionIdException
)
from bs4 import BeautifulSoup
import time
import google_colab_selenium as gs

In [ ]:
sublinks = []
contents = []
htmls = []
tags = []
generalColumnsDict = {
    '观点': "https://www.cankaoxiaoxi.com/#/generalColumns/guandian",
}
for k, v in generalColumnsDict.items():
    cankaoxiaoxi_generalColumns_visiting(sublinks, contents, htmls, tags, k, v, "templateModule")


# save_to_csv(sublinks, contents, htmls, "index_contents.csv", tags)
save_lists_to_csv("index_contents.csv", sublink = sublinks, content = contents, html = htmls, tag = tags)

Initializing Chromedriver


<IPython.core.display.Javascript object>

Enter Page
https://www.cankaoxiaoxi.com/#/generalColumns/guandian
Switching!!


KeyboardInterrupt: 

In [ ]:
# save_lists_to_csv("generalColumns_sublink_contents_temp.csv", sublink = sublinks, content = contents, html = htmls, tag = tags)

CSV saved to /content/drive/MyDrive/Proj/News_fetcher/data/crawled_data/2025-06-19_generalColumns_sublink_contents_temp.csv


In [ ]:
# save_to_csv(sublinks, contents, "generalColumns_sublink_contents.csv", tags)

### simulate


##### index

In [ ]:
from tqdm import tqdm
def cankaoxiaoxi_index_visiting(sublinks, contents, htmls, url="https://www.cankaoxiaoxi.com/#/index"):
    driver = restart_driver_and_get(url)
    wait_for_stable_page(driver)


    num_articles = len(driver.find_elements(By.CLASS_NAME, "clickPart"))
    original_url = driver.current_url
    # Create manual tqdm progress bar
    pbar = tqdm(total=num_articles, desc="Scraping index")
    cnt = 0
    while cnt < num_articles:
        try:
            # print("Enter Index Page")
            original_window = driver.current_window_handle
            cards = driver.find_elements(By.CLASS_NAME, "clickPart")
            # print(driver.current_url)
            cards[cnt].click()

            for handle in driver.window_handles:
                if handle != original_window:
                    # print("Switching!!")
                    driver.switch_to.window(handle)
                    break
            wait_for_stable_page(driver)
            article_url = driver.current_url
            sublinks.append(article_url)
            htmls.append(driver.page_source)
            # print(article_url)
            # Scrape the article
            soup = BeautifulSoup(driver.page_source, "lxml")
            # article = soup.find("div", class_="detailsPage")
            # print(article.get_text(strip=True) if article else "No article found")
            article = soup.get_text()
            print(driver.title)
            print(article)
            contents.append(article)

            # Go back to the list page
            driver.close()
            driver.switch_to.window(original_window)
            # print(driver.current_url)

        except Exception as e:
            print(e)
            try:
                # safe_get(driver, url)
                driver = restart_driver_and_get(url)
                wait_for_stable_page(driver)
            except:
                print(f"[{cnt}] Retry failed. Aborting.")
                break
        cnt += 1
        pbar.update(1)

    pbar.close()
    driver.quit()

用更多新闻来截断
From 首页>新闻详情 to 更多新闻

##### generalColumns

In [ ]:
from tqdm import tqdm

def cankaoxiaoxi_generalColumns_visiting(sublinks, contents, htmls, tags, tag, col_url="https://www.cankaoxiaoxi.com/#/generalColumns/guandian", target_class="templateModule", load_button_clicks=3):
    # driver = get_fresh_driver()
    # safe_get(driver, url)
    driver = restart_driver_and_get(col_url)
    wait_for_stable_page(driver)


    load_button = driver.find_elements(By.CLASS_NAME, "generalColumns-loadMore")
    if load_button:
      load_button = load_button[0]
      for _ in range(load_button_clicks):
        load_button.click()
        wait_for_stable_page(driver)
        load_button = driver.find_elements(By.CLASS_NAME, "generalColumns-loadMore")[0]
        # print(len(driver.find_elements(By.CLASS_NAME, target_class)))

    num_articles = len(driver.find_elements(By.CLASS_NAME, target_class))
    # Create manual tqdm progress bar
    pbar = tqdm(total=num_articles, desc="Scraping index")
    cnt = 0
    while cnt < num_articles:
        try:
            print("Enter Page")
            original_window = driver.current_window_handle
            cards = driver.find_elements(By.CLASS_NAME, target_class)
            print(driver.current_url)
            cards[cnt].click()

            for handle in driver.window_handles:
                if handle != original_window:
                    print("Switching!!")
                    driver.switch_to.window(handle)
                    break
            wait_for_stable_page(driver)
            article_url = driver.current_url
            sublinks.append(article_url)
            tags.append(tag)
            htmls.append(driver.page_source)

            # Scrape the article
            soup = BeautifulSoup(driver.page_source, "lxml")
            # article = soup.find("div", class_="detailsPage")
            # print(article.get_text(strip=True) if article else "No article found")
            article = soup.get_text()
            print(driver.title)
            print(article)
            contents.append(article)


            # Go back to the list page
            driver.close()
            driver.switch_to.window(original_window)
            # print(driver.current_url)

        except Exception as e:
            print(e)
            try:
                # safe_get(driver, url)
                driver = restart_driver_and_get(url)
                wait_for_stable_page(driver)
            except:
                print(f"[{cnt}] Retry failed. Aborting.")
                break
        cnt += 1
        pbar.update(1)

    pbar.close()
    driver.quit()

### post process of contents

In [ ]:
index_contents = load_csv("index_contents.csv")

In [ ]:
index_contents.loc[64,'content']

'参考消息第一关注中国国际观点锐参考体育健康科技应用文化旅游参考漫谈参考智库军事民族品牌参考人物产经地方译名文娱我的位置：首页>文化旅游>新闻详情伊朗媒体：临危受命的伊朗军方新领导层2025-06-17 17:03:4638547分享到：用微信扫描二维码参考消息网6月17日报道 据伊朗光明新闻网6月13日报道，伊朗最高领袖哈梅内伊6月13日颁布一系列法令，任命伊朗伊斯兰革命卫队和武装部队总参谋部新负责人。上述任命是在伊朗武装部队总参谋长穆罕默德·巴盖里、伊朗伊斯兰革命卫队总司令侯赛因·萨拉米于当天凌晨在以色列袭击伊朗首都德黑兰时身亡后作出的。哈梅内伊6月13日发表声明说，等待以色列的必将是“严厉的惩罚”。武装部队总参谋长赛义德·阿卜杜勒—拉希姆·穆萨维据伊朗新闻电视台网站6月13日报道，在伊朗武装部队总参谋长穆罕默德·巴盖里在以色列发动的恐怖袭击中牺牲后，伊斯兰革命领袖哈梅内伊任命赛义德·阿卜杜勒—拉希姆·穆萨维为其继任者。穆萨维自2017年8月起担任伊朗伊斯兰共和国军队总司令，在增强军队实力方面发挥了关键作用。随后，2019年5月，哈梅内伊任命他为哈塔姆安比亚防空基地指挥官，进一步巩固了他在国家军事战略和行动中的核心作用。穆萨维1960年出生于伊朗中部的圣城库姆，毕业于伊玛目阿里军官大学，拥有最高国防大学国防研究博士学位，于1979年加入伊朗军队。20世纪80年代两伊战争期间，穆萨维在炮兵部队前线服役，包括在库尔德斯坦省和胡齐斯坦省。他参与了多项军事行动，是公认的战争老兵。两伊战争结束后，他于1997年完成高级指挥与参谋课程，随后在最高国防大学获得博士学位。1999年至2005年，他担任军队联合参谋部参谋长；2008年至2016年，担任军队副总司令；此后在2016年至2017年期间，担任武装部队副参谋长。在担任上述领导职务之前，穆萨维还曾担任伊玛目阿里军官大学校长，为军事人才培养作出贡献。他还领导过军队的东北作战基地，负责该地区的战略行动。此外，他在陆军担任过训练副部长以及规划与项目副部长，在塑造军事战备和战略方面发挥关键作用。他因在作战领域的专业能力被任命为军队作战部长，后来成为军队战略研究中心主任，从事高级别研究和政策制定工作。伊斯兰革命领袖在任命书中表示，考虑到穆萨维的卓越贡献和宝贵经验，任命其为巴盖里的继任者。伊斯兰革命卫队总司令穆罕默德·帕克普尔据伊朗新

#### try trafilatura Worked!

In [ ]:
!pip install trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 25.9 MB/s eta 0:00:00


In [ ]:
import trafilatura

def clean_contents_and_save(original_file_name):
    df = load_csv(original_file_name)
    # Extract clean text from each HTML
    clean_texts = df['html'].apply(
        lambda x: trafilatura.extract(x, include_comments=False, include_tables=False)
        if isinstance(x, str) and x.strip() else None
    )

    # Drop 'content' and 'html' columns, then add clean_text
    cleaned_df = df.drop(columns=['content', 'html'], errors='ignore').copy()
    cleaned_df['clean_text'] = clean_texts
    # Drop rows with less than 15 tokens in 'clean_text'
    cleaned_df = cleaned_df[cleaned_df['clean_text'].apply(
        lambda x: len(x) >= 15
    )].reset_index(drop=True)

    # Save the cleaned DataFrame to a new CSV file
    # save_lists_to_csv("index_contents.csv", sublink = sublinks, content = contents, html = htmls, tag = tags)
    cleaned_df.to_csv(f'{prefix}data/crawled_data/{suffix}_cleaned_{original_file_name}', index=False, encoding='utf-8-sig')
    print(f"CSV saved to {prefix}data/crawled_data/{suffix}_cleaned_{original_file_name}")
    return cleaned_df


In [ ]:
clean_df = clean_contents_and_save("index_contents.csv")

CSV saved to /content/drive/MyDrive/Proj/News_fetcher/data/crawled_data/2025-06-19_cleaned_index_contents.csv


In [ ]:
clean_df

,Unnamed: 0,sublink,clean_text
0,0,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...,6月17日，我国在酒泉卫星发射中心成功组织实施梦舟载人飞船零高度逃逸飞行试验，标志着我国载人...
1,1,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...,参考消息网6月19日报道 据俄新社6月17日报道，莫斯科旅游委员会主席叶夫根尼·科兹洛夫表示...
2,2,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...,参考消息网6月19日报道 据新西兰广播电台网站6月18日报道，新西兰总理克里斯托弗·拉克森1...
3,3,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...,参考消息网6月19日报道 俄罗斯《消息报》网站6月17日刊登题为《中式外交》的文章，作者是达...
4,4,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...,参考消息网6月18日报道 美国《纽约时报》网站6月16日刊发题为《这个不是特别可爱的小精灵能...
...,...,...,...
66,66,https://www.cankaoxiaoxi.com/#/detailsPage/wen...,参考消息网6月17日报道 澳大利亚“对话”网站近日刊发题为《非洲足球运动员在中国：体育移民趋...
67,67,https://www.cankaoxiaoxi.com/#/detailsPage/wen...,《为人父：爱与权力的历史》一书封面\n参考消息网6月17日报道 英国《经济学人》周刊网站近日...
68,68,https://www.cankaoxiaoxi.com/#/detailsPage/wen...,参考消息网6月17日报道 美国《时代》周刊网站6月7日刊登题为《人工智能无法取代人类的教育，...
69,69,https://www.cankaoxiaoxi.com/#/detailsPage/wen...,参考消息网6月17日报道 西班牙《国家报》网站近日刊登题为《“积读”现象：我们是如何对只囤不...


#### clickpart 不行，他是用event listener处理的 需要simulate

In [ ]:
cards = driver.find_elements(By.CLASS_NAME, "clickPart")
for card in cards:
    print(card)
    onclick = card.get_attribute("onclick")
    if onclick:
        print("onclick JS:", onclick)
for card in soup.select(".clickPart"):
    print(1)
    a = card.find("a")
    if a and a.get("href"):
        print("Title:", card.select_one(".content_title").text.strip())
        print("URL:", a["href"])

# LLM Tag Formation

### tongyi qianwen

In [ ]:
def make_prompt_cn(news):
    return f"""你是一个新闻分类助手，请根据以下新闻内容为其生成几个合适的主题标签。 你可以从以下标签中选择也可以自己生成合适的标签：政治、体育、财经、科技、健康、教育、国际、娱乐、环境、社会。

新闻内容：
\"\"\"{news}\"\"\"

请输出中文标签，以井号分隔：
标签：
"""

In [ ]:
# def make_prompt_rating_cn(news):
#     return f"""你是一个新闻分类助手，请先根据以下新闻内容为其生成几个合适的主题标签。 你可以从以下标签中选择也可以自己生成合适的标签：政治、体育、财经、科技、健康、教育、国际、娱乐、环境、社会。然后对此新闻质量进行打分，分为0到10分。

# 新闻内容：
# \"\"\"{news}\"\"\"

# 请输出中文标签和阿拉伯数字分数，全部用井号分隔：
# 例子：#标签 #标签 #分数
# ：
# """

In [ ]:
api_key = "sk-3ac60781289448f782c6398ac389d3ff"

In [ ]:
cleaned_index_contents = load_csv("cleaned_index_contents.csv").loc[:,["sublink", "clean_text"]]

In [ ]:
articles = load_csv("cleaned_index_contents.csv").loc[:,'clean_text'].to_list()


AttributeError: 'DataFrame' object has no attribute 'to_list'

In [ ]:
# article = "参考消息第一关注中国国际观点锐参考体育健康科技应用文化旅游参考漫谈参考智库军事民族品牌参考人物产经地方译名文娱我的位置：首页>新闻详情美媒：中国中亚多领域合作蒸蒸日上2025-06-17 17:53:0144411分享到：用微信扫描二维码参考消息网6月17日报道 彭博新闻社网站6月16日刊登题为《习近平利用对哈萨克斯坦的访问推动对中亚的地区影响力》的文章，作者是纳里曼·吉齐季诺夫。主要内容编译如下：中国国家主席习近平将前往哈萨克斯坦，并在该国与中亚领导人举行会谈。中亚是一个处于华盛顿与北京利益竞争核心交汇点的广袤地区。习近平的此次访问将提出与七国集团(G7)峰会不同的议题。习近平将于16日会晤哈萨克斯坦总统卡瑟姆若马尔特·托卡耶夫，并于次日与哈萨克斯坦、吉尔吉斯斯坦、塔吉克斯坦、土库曼斯坦和乌兹别克斯坦领导人共同出席第二届中国—中亚峰会。在哈萨克斯坦首都阿斯塔纳举行的这场峰会与加拿大主办的G7峰会同期举行。两年前习近平曾在中国城市西安以东道主身份主持了首届中国—中亚峰会。英国棱镜战略情报公司合伙人凯特·马林森说，习近平访问哈萨克斯坦的一个关键目标是为中美经贸摩擦加剧做好“未来防范”。她说：“过去三年，中国在中亚的影响力显著增强……中国继续致力于守护其经济和供应链免受未来与美国的任何对抗的影响。”在习近平加强与中亚国家的友好关系之际，美国总统特朗普的关税战很可能成为重要议题——大国在中亚地区的影响力竞争已经愈演愈烈。拥有丰富自然资源的中亚地区对于中国的意义正变得日益重要。这一地区还为中国提供向欧洲输送货物的陆路通道。这将是中国领导人自2022年以来第三次访问哈萨克斯坦。中共中央对外联络部部长刘建超在今年3月的一次讲话中表示，中国的发展需要一个蒸蒸日上的中亚，中国也欢迎中亚国家搭乘中国高质量发展、高水平开放快车。中国一直加强与全球南方的合作。近几个月，习近平出席了多个区域性组织的峰会，利用与拉美和非洲国家领导人的会晤推动建立更紧密的关系。与此同时，中国与中亚地区的经济合作已经取得明显进展，在双方贸易量大增的同时，也把投资扩张到从能源到铁路等各个领域。自2023年以来，中国已成为哈萨克斯坦最大贸易伙伴。随着华为、比亚迪等中国企业拓展在当地的市场份额，中国与中亚五国的进出口总额已连续数年录得高速增长。随着中国和中亚五国领导人举行会晤，更多商业协议很可能将被提交讨论。（编译/曹卫国）2025年5月29日，中国—中亚国际人文旅游专列（西安—阿拉木图段）从陕西西安首发开行。（杨一苗 摄）更多新闻美媒：中国赢得全球民心的“最新武器”2025-06-18 09:47摩根士丹利：中国机器人市场规模到2028年将翻一番2025-06-18 09:45塔吉克斯坦媒体：北京直飞杜尚别首条航线开通2025-06-18 09:39哈萨克斯坦智库：中国中亚携手打造合作新模式2025-06-18 09:41美媒：中国中亚多领域合作蒸蒸日上2025-06-17 17:53外媒评述：中国深谙“为邻之道”，至今仍行之有效2025-06-17 17:52英伟达将首次来华参加链博会2025-06-17 14:255月访港旅客同比上升两成2025-06-17 09:53美媒：中国正重塑全球药物创新格局2025-06-16 12:15外媒：中拉携手迈向命运共同体2025-06-16 08:59点击加载更多专题 2025企业品牌国际传播论坛 二战回眸与反思 第十五届北京国际电影节 2025世界看两会 聚焦俄乌冲突三周年排行榜外媒分析：以色列为何此时对伊朗“动手”美国：两场“大戏”即将同时上场！外媒：伊朗和以色列军队孰强孰弱？突发！以色列对伊朗发动打击美媒：美政府取消四国数十万移民合法身份中国成功斡旋，“化解宿敌”美媒：印航波音客机坠毁原因何在？伊朗：“友好”国家已就以色列动武向德黑兰发出警报针对以色列袭击伊朗，国际社会纷纷发声西媒：论中国的长远思维与五年规划关于参考消息|版权声明|广告服务|联系我们国新网备2012001   互联网新闻信息服务许可证  京ICP备11013708  京公网安备11040202440054  互联网药品信息服务资格证书(京)-非经营性-2019-0121  广播电视节目制作经营许可证  中华人民共和国增值电信业务经营许可证  高新技术企业证书举报电话：010-63071251    举报邮箱：ckxx@xinhua.org- 参考消息报社 版权所有 - 站长统计"


In [ ]:
def get_tags_and_rating(article):
    # First: ask for tags
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": make_prompt_cn(article)}
    ]

    try:
        tag_response = client.chat.completions.create(
            model="qwen-turbo",
            messages=messages,
        )

        tag_content = tag_response.choices[0].message.content
        usage_tags = tag_response.usage

        # Second: ask for rating based on same conversation
        messages.append({"role": "assistant", "content": tag_content})
        messages.append({"role": "user", "content": "请你根据文章的内容以及你刚才的打标签结果，为这篇文章打一个总分，满分10分。"})

        rating_response = client.chat.completions.create(
            model="qwen-turbo",
            messages=messages,
        )

        rating_content = rating_response.choices[0].message.content
        usage_rating = rating_response.usage

        return {
            "tags": tag_content,
            "rating": rating_content,
            "input_tokens": usage_tags.prompt_tokens + usage_rating.prompt_tokens,
            "output_tokens": usage_tags.completion_tokens + usage_rating.completion_tokens,
            "total_tokens": usage_tags.total_tokens + usage_rating.total_tokens,
        }

    except Exception as e:
        print(f"Error: {e}")
        return {
            "tags": None,
            "rating": None,
            "input_tokens": None,
            "output_tokens": None,
            "total_tokens": None,
        }

In [ ]:
# import os
# from openai import OpenAI

# try:
#     client = OpenAI(
#         # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
#         api_key=api_key,
#         base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
#     )

#     completion = client.chat.completions.create(
#         # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
#         model="qwen-turbo",
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant."},
#             {"role": "user", "content": make_prompt_cn(article)},
#         ],
#     )
#     print(completion.choices[0].message.content)
#     print(completion.choices)
# except Exception as e:
#     print(f"错误信息：{e}")
#     print("请参考文档：https://help.aliyun.com/zh/model-studio/developer-reference/error-code")

#旅游 #国际 #文化交流 #8
[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='#旅游 #国际 #文化交流 #8', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))]


In [ ]:
# import os
# from openai import OpenAI

# try:
#     client = OpenAI(
#         # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
#         api_key=api_key,
#         base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
#     )

#     completion = client.chat.completions.create(
#         # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
#         model="qwen-turbo",
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant in news understanding aspect."},
#             {"role": "user", "content": make_prompt_cn(article)},
#         ],
#     )
#     print(completion.choices[0].message.content)
#     print(completion.choices)
# except Exception as e:
#     print(f"错误信息：{e}")
#     print("请参考文档：https://help.aliyun.com/zh/model-studio/developer-reference/error-code")

旅游 # 国际 # 文化 # 社会
[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='旅游 # 国际 # 文化 # 社会', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))]


##### gpt method for one by one

In [ ]:
# import os
# import pandas as pd
# from openai import OpenAI
# from time import sleep

# # Load article texts
# original_file_name = "index_contents.csv"
# df = load_csv(f"cleaned_{original_file_name}")
# articles = df["clean_text"].dropna().tolist()[:2]

# # Setup Qwen API client
# client = OpenAI(
#     api_key=api_key,  # or api_key="sk-xxx"
#     base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
# )

# # Prompt builder
# def make_messages(article):
#     return [
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": make_prompt_rating_cn(article)},
#     ]

# # Store results
# results = []

# for i, article in enumerate(articles, start=1):
#     try:
#         messages = make_messages(article)
#         response = client.chat.completions.create(
#             model="qwen-turbo",
#             messages=messages,
#         )

#         output = response.choices[0].message.content
#         usage = response.usage

#         results.append({
#             "index": i,
#             "article": article,
#             "tags": output,
#             "input_tokens": usage.prompt_tokens,
#             "output_tokens": usage.completion_tokens,
#             "total_tokens": usage.total_tokens,
#         })

#         print(f"[{i}/{len(articles)}] ✅ done: input {usage.prompt_tokens}, output {usage.completion_tokens}")

#     except Exception as e:
#         print(f"[{i}] ❌ Error: {e}")
#         results.append({
#             "index": i,
#             "article": article,
#             "tags": None,
#             "input_tokens": None,
#             "output_tokens": None,
#             "total_tokens": None,
#         })
#         sleep(2)  # optional cooldown after error

# # Save to DataFrame and CSV
# results_df = pd.DataFrame(results)
# results_df.to_csv("tagged_articles_with_tokens.csv", index=False, encoding="utf-8-sig")
# results_df.to_csv(f'{prefix}data/crawled_data/{suffix}_taged_{original_file_name}', index=False, encoding='utf-8-sig')
# print(f"🔽 saved to {prefix}data/crawled_data/{suffix}_taged_{original_file_name}")


[1/2] ✅ done: input 653, output 8
[2/2] ✅ done: input 468, output 10
🔽 saved to /content/drive/MyDrive/Proj/News_fetcher/data/crawled_data/2025-06-19_taged_index_contents.csv


###### add rating 看似 work

In [ ]:
import os
import pandas as pd
from openai import OpenAI
from time import sleep

# Prompt builder
def make_tag_prompt(article):
    return [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": make_prompt_cn(article)},
    ]

def make_rating_followup(messages_so_far, tag_response):
    return messages_so_far + [
        {"role": "assistant", "content": tag_response},
        {"role": "user", "content": "请你给这篇新闻的质量打分，满分10分。直接输出数字"},
    ]

def tagging_and_rating(original_file_name, model_name="qwen-turbo"):
    df = load_csv(f"cleaned_{original_file_name}")
    articles = df["clean_text"].tolist()[:]  # limit for testing
    urls = df["sublink"].tolist()[:]

    # Setup Qwen API client
    client = OpenAI(
        api_key=api_key,  # or api_key="sk-xxx"
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )

    # Store results
    results = []

    for i, article in enumerate(articles, start=1):
        try:
            # Step 1: Ask for tags
            messages = make_tag_prompt(article)
            tag_response = client.chat.completions.create(
                model=model_name,
                messages=messages,
            )
            tag_text = tag_response.choices[0].message.content
            tag_usage = tag_response.usage

            # Step 2: Ask for rating (continue chat)
            rating_messages = make_rating_followup(messages, tag_text)
            rating_response = client.chat.completions.create(
                model=model_name,
                messages=rating_messages,
            )
            rating_text = rating_response.choices[0].message.content
            rating_usage = rating_response.usage

            # Combine token usage
            total_input_tokens = tag_usage.prompt_tokens + rating_usage.prompt_tokens
            total_output_tokens = tag_usage.completion_tokens + rating_usage.completion_tokens
            total_tokens = tag_usage.total_tokens + rating_usage.total_tokens

            # Save result
            results.append({
                "index": i,
                "article": article,
                "tags": tag_text,
                "rating": rating_text,
                "input_tokens": total_input_tokens,
                "output_tokens": total_output_tokens,
                "total_tokens": total_tokens,
                "url": urls[i-1],
            })

            print(f"[{i}/{len(articles)}] ✅ tagged & rated: input {total_input_tokens}, output {total_output_tokens}")

        except Exception as e:
            print(f"[{i}] ❌ Error: {e}")
            results.append({
                "index": i,
                "article": article,
                "tags": None,
                "rating": None,
                "input_tokens": None,
                "output_tokens": None,
                "total_tokens": None,
                "url": None,
            })
            sleep(2)  # optional cooldown

    # Save to DataFrame and CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv("tagged_articles_with_tokens.csv", index=False, encoding="utf-8-sig")
    results_df.to_csv(f'{prefix}data/crawled_data/{suffix}_tagged_{original_file_name}', index=False, encoding='utf-8-sig')
    print(f"🔽 saved to {prefix}data/crawled_data/{suffix}_tagged_{original_file_name}")


In [ ]:
original_file_name = "index_contents.csv"
tagging_and_rating(original_file_name)

[1/71] ✅ tagged & rated: input 1290, output 17
[2/71] ✅ tagged & rated: input 910, output 7
[3/71] ✅ tagged & rated: input 1394, output 11
[4/71] ✅ tagged & rated: input 1290, output 13
[5/71] ✅ tagged & rated: input 1399, output 18
[6/71] ✅ tagged & rated: input 1270, output 17
[7/71] ✅ tagged & rated: input 865, output 16
[8/71] ✅ tagged & rated: input 4073, output 32
[9/71] ✅ tagged & rated: input 1735, output 10
[10/71] ✅ tagged & rated: input 1735, output 10
[11/71] ✅ tagged & rated: input 1638, output 11
[12/71] ✅ tagged & rated: input 1683, output 10
[13/71] ✅ tagged & rated: input 1706, output 11
[14/71] ✅ tagged & rated: input 1290, output 17
[15/71] ✅ tagged & rated: input 912, output 9
[16/71] ✅ tagged & rated: input 539, output 6
[17/71] ✅ tagged & rated: input 539, output 6
[18/71] ✅ tagged & rated: input 859, output 10
[19/71] ✅ tagged & rated: input 461, output 10
[20/71] ✅ tagged & rated: input 719, output 10
[21/71] ✅ tagged & rated: input 1688, output 11
[22/71] ✅ tag

In [ ]:
df = load_csv("tagged_index_contents.csv")
df

,index,article,tags,rating,input_tokens,output_tokens,total_tokens,url
0,1,6月17日，我国在酒泉卫星发射中心成功组织实施梦舟载人飞船零高度逃逸飞行试验，标志着我国载人...,科技 # 航天 # 探月工程 # 国际竞争,9,1290,17,1307,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...
1,2,参考消息网6月19日报道 据俄新社6月17日报道，莫斯科旅游委员会主席叶夫根尼·科兹洛夫表示...,#旅游 #国际 #文化,8,910,7,917,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...


##### clean

##### gpt method for batch. Not sure if this works

In [ ]:
import pandas as pd, json, os
from pathlib import Path

# 1) load the cleaned articles
df = load_csv("cleaned_index_contents.csv")
articles = df.loc[:,'clean_text'].to_list()[:2]

In [ ]:
articles

['6月17日，我国在酒泉卫星发射中心成功组织实施梦舟载人飞船零高度逃逸飞行试验，标志着我国载人月球探测工程研制工作取得新的重要突破。（新华社发）\n参考消息网6月19日报道 据法哈拉斯新闻网6月17日报道，中国正通过太空技术领域的重大进展推进其探月雄心。据中国载人航天工程办公室消息，中国17日成功组织实施梦舟载人飞船零高度逃逸飞行试验，这是其实现航天员登月目标的关键一步。\n报道称，此次测试涉及飞船的逃逸塔系统，通过点燃固体火箭发动机使船塔组合体升空。最终，梦舟载人飞船安全返回地面，验证了其系统安全性。这是否意味着这一次中国将先于美国国家航空航天局(NASA)登月？\n报道称，中国梦舟载人飞船零高度逃逸飞行试验的成功，标志着其月球探索进程中的一个关键节点，使其有望与NASA的阿耳忒弥斯登月计划展开竞争。\n报道称，中国在月球技术领域的进步凸显了太空探索领域的竞争格局，中美两国正力争在未来任务中占据主导地位。\n另据印度新闻9直播网站6月17日报道，中国成功进行梦舟载人飞船发射台中止试验。这是在酒泉卫星发射中心进行的零高度逃逸飞行试验。\n报道称，这是继1998年神舟载人飞船零高度逃逸飞行试验之后，时隔27年中国再度组织实施这项试验。神舟飞船用于中国空间站乘组人员轮换。\n经过重新设计的梦舟载人飞船逃逸发动机正常运行，船塔组合体在固体发动机推进下升空，返回舱与逃逸塔安全分离。在飞行两分钟后，返回舱使用气囊缓冲方式安全着陆于试验落区预定区域。\n报道指出，这一安全机制对载人航天至关重要，因为一旦飞行过程中发生故障，它可以使乘员迅速远离危险并安全返回地面。\n报道称，此次试验的成功为中国后续载人月球探测任务奠定了重要的技术基础。中国的载人月面着陆器被命名为“揽月”。\n由于大气压力的变化，乘组人员的逃逸设备会根据不同的高度进行调整。乘组人员逃逸系统的火箭发动机位于飞船的顶部。梦舟载人飞船系统全面负责逃逸与救生两项任务。\n报道称，中国一直在测试其地月空间运输体系各个组成部分，并加强了载人登月计划。（编译/邬眉、马丹）',
 '参考消息网6月19日报道 据俄新社6月17日报道，莫斯科旅游委员会主席叶夫根尼·科兹洛夫表示，中国人在莫斯科旅游的日均消费在所有外国游客群体中位居第一，人均单日支出约4.4万卢布(1卢布约合0.09元人民币——本网注)。\n科兹洛夫说：“外国游客对

In [ ]:
import pandas as pd, json, os
from pathlib import Path

# 1) load the cleaned articles
df = load_csv("cleaned_index_contents.csv")
articles = df.loc[:,'clean_text'].to_list()[:2]

# 2) build one JSON record per article
def make_messages(text: str):
    return [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": make_prompt_cn(text)}   # <-- your prompt fn
    ]

out_path = Path("qwen_batch_input.jsonl")
with out_path.open("w", encoding="utf-8") as f:
    for i, art in enumerate(articles, start=1):
        req = {
            "custom_id": str(i),        # any unique ID you like
            "method":   "POST",
            "url":      "/v1/chat/completions",
            "body": {
                "model": "qwen-turbo",
                "messages": make_messages(art)
            }
        }
        f.write(json.dumps(req, ensure_ascii=False, separators=(",", ":")) + "\n")


Upload file

In [ ]:
from openai import OpenAI
import time, os

client = OpenAI(
    api_key=api_key,   # or api_key="sk-xxx"
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 1) upload
file_id = client.files.create(file=out_path, purpose="batch").id
print("uploaded file:", file_id)

# 2) start batch (24 h window is cheapest)
batch = client.batches.create(
    input_file_id=file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h"      # fixed price tier
)
print("batch id:", batch.id)


uploaded file: file-batch-18404b43fd744f1a98a62687
batch id: batch_0184018f-5b72-4041-8332-33ef6a2021ce


poll until finished

In [ ]:
while batch.status not in {"completed", "failed", "expired", "cancelled"}:
    time.sleep(10)
    batch = client.batches.retrieve(batch.id)
    print("status:", batch.status)

if batch.status != "completed":
    raise RuntimeError(f"Batch ended with status {batch.status}: {batch.errors}")


status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress


KeyboardInterrupt: 

Download results (JSONL again)

In [ ]:
out_jsonl = Path("qwen_batch_result.jsonl")
client.files.content(batch.output_file_id).write_to_file(out_jsonl)
print("results saved to", out_jsonl)


merge back

In [ ]:
rows = []
with out_jsonl.open(encoding="utf-8") as f:
    for line in f:
        rec = json.loads(line)
        cid = int(rec["custom_id"])          # original row number (1-based)
        text = rec["response"]["body"]["choices"][0]["message"]["content"]
        rows.append((cid, text))

tags_df = pd.DataFrame(rows, columns=["row", "tags"]).set_index("row")
final_df = df.join(tags_df, how="left")
final_df.to_csv("tagged_articles.csv", index=False, encoding="utf-8-sig")


### LLAMA 3.2 instruct 3B

In [ ]:
!pip install -U transformers

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

In [ ]:
model = "meta-llama/Llama-3.2-3B-Instruct"

In [ ]:
from transformers import pipeline
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-3B-Instruct")

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
def make_prompt(article):
    return f"""You are a news classifier. Your job is to assign topic tags to news articles.
Choose only from the following tags or create ur own ones: [Politics, Sports, Business, Technology, Health, Education, Crime, Environment, Entertainment, International].

Article:
\"\"\"{article}\"\"\"

Output the tags as a Python list:
Tags:"""

In [ ]:
messages = [
    {"role": "user", "content": "Who are you?"},
]

In [ ]:
# pipe = pipeline("chat", model="meta-llama/Llama-3.2-3B-Instruct", device_map="auto")

# High-level role-based input
response = pipe([
    {"role": "user", "content": "Tag this news article: A football match between Spain and Germany ended in a draw."}
])

In [ ]:
article = "参考消息第一关注中国国际观点锐参考体育健康科技应用文化旅游参考漫谈参考智库军事民族品牌参考人物产经地方译名文娱我的位置：首页>新闻详情塔吉克斯坦媒体：北京直飞杜尚别首条航线开通2025-06-18 09:39:1448028分享到：用微信扫描二维码参考消息网6月18日报道 据塔吉克斯坦阿维斯塔通讯社网站6月16日报道，首条连接塔吉克斯坦首都杜尚别与中国首都北京的直飞航线于16日正式开通。报道称，航线开通仪式16日上午在北京大兴国际机场举行，这一事件开启了塔中两国运输和实业合作发展的重要阶段。该航线的首飞航班由中国南方航空安排波音787机型飞机执飞。未来该航线计划采用波音737-8型飞机执飞，每周安排两班，时间分别为周二和周六。塔吉克斯坦政府民航部门负责人哈比布洛·纳扎尔佐达称，这一航线的开通具有历史意义。他强调，这对进一步巩固塔中双边关系和发展两国航空运输十分重要。纳扎尔佐达说：“这条航线不仅是旅客运输通道，也是我们两国间相互信任、扩大合作和文化交流的象征。”中国南方航空副总经理曲光吉指出，新航线的开通将架起中国和塔吉克斯坦之间更加便捷的空中桥梁，有力促进双边经贸往来、人文交流和旅游合作，为两国关系发展和区域互联互通注入新的强劲动能。报道称，首航航班机舱内部以中式风格装饰，搭配刺绣和灯笼等中国传统饰物。机上不仅提供美味餐食，还为旅客组织了有奖问答等娱乐活动。（编译/刘洋）6月16日，CZ6051航班从北京大兴国际机场起飞。（新华社发）更多新闻美媒：中国赢得全球民心的“最新武器”2025-06-18 09:47摩根士丹利：中国机器人市场规模到2028年将翻一番2025-06-18 09:45塔吉克斯坦媒体：北京直飞杜尚别首条航线开通2025-06-18 09:39哈萨克斯坦智库：中国中亚携手打造合作新模式2025-06-18 09:41美媒：中国中亚多领域合作蒸蒸日上2025-06-17 17:53外媒评述：中国深谙“为邻之道”，至今仍行之有效2025-06-17 17:52英伟达将首次来华参加链博会2025-06-17 14:255月访港旅客同比上升两成2025-06-17 09:53美媒：中国正重塑全球药物创新格局2025-06-16 12:15外媒：中拉携手迈向命运共同体2025-06-16 08:59点击加载更多专题 2025企业品牌国际传播论坛 二战回眸与反思 第十五届北京国际电影节 2025世界看两会 聚焦俄乌冲突三周年排行榜外媒分析：以色列为何此时对伊朗“动手”美国：两场“大戏”即将同时上场！外媒：伊朗和以色列军队孰强孰弱？突发！以色列对伊朗发动打击美媒：美政府取消四国数十万移民合法身份中国成功斡旋，“化解宿敌”美媒：印航波音客机坠毁原因何在？伊朗：“友好”国家已就以色列动武向德黑兰发出警报针对以色列袭击伊朗，国际社会纷纷发声西媒：论中国的长远思维与五年规划关于参考消息|版权声明|广告服务|联系我们国新网备2012001   互联网新闻信息服务许可证  京ICP备11013708  京公网安备11040202440054  互联网药品信息服务资格证书(京)-非经营性-2019-0121  广播电视节目制作经营许可证  中华人民共和国增值电信业务经营许可证  高新技术企业证书举报电话：010-63071251    举报邮箱：ckxx@xinhua.org- 参考消息报社 版权所有 - 站长统计"
response = pipe([
    {"role": "user", "content": make_prompt(article)}
])

# 3. sort

### gpt one

In [ ]:
def search_by_tag(df, tag, top_k=None):
    """
    Return news containing the specified tag, sorted by rating descending.

    Args:
        df (pd.DataFrame): The DataFrame with 'tags', 'clean_text', 'url', and 'rating' columns.
        tag (str): The tag to search for (e.g., "科技").
        top_k (int or None): Return only top k results, or all if None.

    Returns:
        pd.DataFrame: Filtered and sorted result with 'url', 'clean_text', and 'rating'.
    """
    if 'rating' not in df.columns:
        raise ValueError("The DataFrame must contain a 'rating' column.")
    if 'tags' not in df.columns or 'article' not in df.columns or 'url' not in df.columns:
        raise ValueError("Missing required columns: 'tags', 'article', or 'url'.")

    # Normalize tags and check if the desired tag is present
    filtered = df[df['tags'].apply(lambda x: isinstance(x, str) and tag in x)]

    # Convert rating to numeric if it's not already
    filtered['rating'] = pd.to_numeric(filtered['rating'], errors='coerce')

    # Sort by rating descending
    filtered = filtered.sort_values(by='rating', ascending=False)

    # Optional: limit to top_k
    if top_k:
        filtered = filtered.head(top_k)

    return filtered[['url', 'article', 'rating']].reset_index(drop=True)


In [ ]:
df = load_csv("tagged_index_contents.csv")

In [ ]:
df

,index,article,tags,rating,input_tokens,output_tokens,total_tokens,url
0,1,6月17日，我国在酒泉卫星发射中心成功组织实施梦舟载人飞船零高度逃逸飞行试验，标志着我国载人...,科技 # 航天 # 探月工程 # 国际竞争,9,1290,17,1307,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...
1,2,参考消息网6月19日报道 据俄新社6月17日报道，莫斯科旅游委员会主席叶夫根尼·科兹洛夫表示...,#旅游 #国际 #文化交流,8,910,7,917,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...
2,3,参考消息网6月19日报道 据新西兰广播电台网站6月18日报道，新西兰总理克里斯托弗·拉克森1...,政治 # 国际,9,1389,6,1395,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...
3,4,参考消息网6月19日报道 俄罗斯《消息报》网站6月17日刊登题为《中式外交》的文章，作者是达...,国际 # 政治 # 经济 # 一带一路,8,1290,13,1303,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...
4,5,参考消息网6月18日报道 美国《纽约时报》网站6月16日刊发题为《这个不是特别可爱的小精灵能...,#国际 #社会 #文化 #经济,8,1390,9,1399,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...
5,6,参考消息网6月18日报道 据香港《南华早报》网站6月16日报道，根据美国摩根士丹利公司16日...,科技 # 工业 # 制造业 # 人工智能 # 机器人,9,1270,17,1287,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...
6,7,参考消息网6月18日报道 据塔吉克斯坦阿维斯塔通讯社网站6月16日报道，首条连接塔吉克斯坦首...,国际 # 交通 # 文化交流,9,858,9,867,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...
7,8,第一关注\n中国\n国际\n观点\n锐参考\n体育健康\n科技应用\n文化旅游\n参考漫谈\...,#民族品牌 #科技应用 #体育健康 #国际 #社会,10,4055,15,4070,https://www.cankaoxiaoxi.com/#/generalColumns/...
8,9,参考消息网6月19日报道 法国《回声报》网站6月16日刊登记者拉斐尔·巴莱涅里撰写的题为《在...,环境 # 社会 # 科技,9,1735,10,1745,https://www.cankaoxiaoxi.com/#/detailsPage/zho...
9,10,参考消息网6月19日报道 法国《回声报》网站6月16日刊登记者拉斐尔·巴莱涅里撰写的题为《在...,环境 # 社会 # 国际,8,1735,10,1745,https://www.cankaoxiaoxi.com/#/detailsPage/zho...


In [ ]:
search_by_tag(df, "科技")

/tmp/ipython-input-162-119500636.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['rating'] = pd.to_numeric(filtered['rating'], errors='coerce')


,url,article,rating
0,https://www.cankaoxiaoxi.com/#/detailsPage/kej...,参考消息网6月19日报道 西班牙《趣味》月刊网站5月14日发表克里斯蒂安·佩雷斯的文章《史上...,10
1,https://www.cankaoxiaoxi.com/#/detailsPage/can...,参考消息网5月21日报道 参考消息报社参考智库日前发布《中国百城海外传播力月度分析报告（20...,10
2,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...,6月17日，我国在酒泉卫星发射中心成功组织实施梦舟载人飞船零高度逃逸飞行试验，标志着我国载人...,9
3,https://www.cankaoxiaoxi.com/#/detailsPage/zho...,参考消息网6月19日报道 美国《福布斯》杂志网站6月16日刊发祖德·克莱门特撰写的文章，题为...,9
4,https://www.cankaoxiaoxi.com/#/detailsPage/zho...,参考消息网6月19日报道 法国《回声报》网站6月16日刊登记者拉斐尔·巴莱涅里撰写的题为《在...,9
5,https://www.cankaoxiaoxi.com/#/detailsPage/zho...,6月17日，我国在酒泉卫星发射中心成功组织实施梦舟载人飞船零高度逃逸飞行试验，标志着我国载人...,9
6,https://www.cankaoxiaoxi.com/#/detailsPage/can...,参考消息网5月14日报道 参考消息报社参考智库日前发布《中国百企海外传播力月度分析报告（20...,9
7,https://www.cankaoxiaoxi.com/#/detailsPage/kej...,参考消息网6月19日报道 据《西班牙人报》网站6月12日报道，通过把六个不同国家的光学时钟连...,9
8,https://www.cankaoxiaoxi.com/#/detailsPage/kej...,参考消息网6月18日报道 美国趣味科学网站5月12日发表弗·麦克杜格尔的文章《行星真的能发展...,9
9,https://www.cankaoxiaoxi.com/#/detailsPage/tt/...,参考消息网6月18日报道 据香港《南华早报》网站6月16日报道，根据美国摩根士丹利公司16日...,9


In [ ]:
search_by_tag(df, "科技").loc[1,'url']

/tmp/ipython-input-162-119500636.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['rating'] = pd.to_numeric(filtered['rating'], errors='coerce')


'https://www.cankaoxiaoxi.com/#/detailsPage/cankaozk/cc58fa8084474bff82014113ece41410/1/2025-05-21%2009:22?childrenAlias=undefined'